In [1]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import load

# 假設你已經載入並預處理 WESAD 數據
# X 為 (samples, timesteps, features)，這裡以 EDA 信號為例
# y 為壓力狀態類別


# 載入資料
wesad = load.WESAD()
df = wesad.separate_and_feature_extract(sample_n=420000)

Using ready-made DataFrame


<Figure size 1000x600 with 0 Axes>

In [2]:
df

array([[[1, 'S10', 0.021379324848018657, ..., 0.10544673374720982,
         0.11477304250001907, 34.29309844970703],
        [1, 'S10', 0.0214518810439985, ..., 0.023563058035714287,
         0.12045248597860336, 34.2890510559082],
        [1, 'S10', 0.02136242519120789, ..., 0.03946184430803571,
         0.11820656806230545, 34.28973388671875],
        ...,
        [1, 'S10', 0.02147831553250883, ..., 0.11543361118861607,
         0.11801749467849731, 34.29108428955078],
        [1, 'S10', 0.021757230706195046, ..., 0.04965449741908482,
         0.11596771329641342, 34.291866302490234],
        [1, 'S10', 0.021459099959067588, ..., 0.06472930908203126,
         0.11862567812204361, 34.29092025756836]],

       [[1, 'S11', 0.012037593266755789, ..., 0.0012320382254464286,
         0.2988918125629425, 34.36301040649414],
        [1, 'S11', 0.011879841876310637, ..., 0.07897099086216518,
         0.29830145835876465, 34.35700988769531],
        [1, 'S11', 0.012031829853284569, ..., 0.010

In [3]:
y = df[:, 0, 0]  # 取得 `label`
X = df[:, :, 2:]  # 移除 `label`,'subject'

In [4]:
X.shape

(30, 59, 18)

In [5]:
y.shape

(30,)

In [6]:
df.shape

(30, 59, 20)

In [7]:
X

array([[[0.021379324848018657, 0.8897033138786044, 0.06319774148725227,
         ..., 0.10544673374720982, 0.11477304250001907,
         34.29309844970703],
        [0.0214518810439985, 0.8892949994632176, 0.06258256514570928,
         ..., 0.023563058035714287, 0.12045248597860336,
         34.2890510559082],
        [0.02136242519120789, 0.8895389994893755, 0.06269687220814518,
         ..., 0.03946184430803571, 0.11820656806230545,
         34.28973388671875],
        ...,
        [0.02147831553250883, 0.8894220858131137, 0.062214745045433374,
         ..., 0.11543361118861607, 0.11801749467849731,
         34.29108428955078],
        [0.021757230706195046, 0.8891831139326095, 0.06327834093839158,
         ..., 0.04965449741908482, 0.11596771329641342,
         34.291866302490234],
        [0.021459099959067588, 0.889101913537298, 0.06377191756928653,
         ..., 0.06472930908203126, 0.11862567812204361,
         34.29092025756836]],

       [[0.012037593266755789, 0.8782380006313

In [8]:
y=y-1

In [9]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=object)

In [10]:
import numpy as np

X = np.array(X, dtype=np.float32)

y = np.array(y, dtype=np.int32)

print("X dtype:", X.dtype)  # 應該是 float32
print("y dtype:", y.dtype)  # 應該是 int32

X dtype: float32
y dtype: int32


In [11]:
# 分割訓練測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train.shape

(24, 59, 18)

In [13]:
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv1D(filters=32, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model


input_shape = (X_train.shape[1], X_train.shape[2]) #(59,18)
num_classes = len(np.unique(y_train))

print(input_shape)
print(num_classes)

# 初始化模型
model = build_cnn_model(input_shape, num_classes)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
epochs = 30
batch_size = 16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# 評估模型
loss, acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {acc * 100:.2f}%')


(59, 18)
2


c:\tensorflow\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 59, 32)         │         2,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 29, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 29, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 14, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        57,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,458 (372.88 KB)

 Trainable params: 95,458 (372.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.6250 - loss: 1.3776 - val_accuracy: 0.6667 - val_loss: 0.6997
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5764 - loss: 0.8568 - val_accuracy: 0.3333 - val_loss: 1.1197
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5764 - loss: 0.8912 - val_accuracy: 0.3333 - val_loss: 1.2863
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5764 - loss: 1.1763 - val_accuracy: 0.5000 - val_loss: 0.6901
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5000 - loss: 0.8854 - val_accuracy: 0.6667 - val_loss: 0.5548
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4931 - loss: 0.8423 - val_accuracy: 0.6667 - val_loss: 0.5499
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5556 - loss: 0.7649 - val_accuracy: 0.8333 - val_loss: 0.5470
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4444 - loss: 0.7371 - val_accuracy: 0.6667 - val_loss: 0.5708